In [1]:
import networkx as nx
import plotly.graph_objects as go
import json
import os
import numpy as np

In [2]:
# Lê todos os arquivos

data = []
for file in os.listdir( '.' ):
    filename = os.fsdecode( file )
    if filename.endswith( '.json' ):
        with open( filename ) as f:
            data.append( json.load( f ) )

In [3]:
# Lê todas as turmas que foram escolhidas

turmas = []
for d in data: # Cada formulário
    for i in d: # Só tem o próprio docente
        turmas += d[ i ][ 'formularios' ]
turmas = list( set( turmas ) )
turmas.sort()

# Apenas os códigos

cods = list( set( [ t[ : -2 ] for t in turmas ] ) )
cods.sort()

In [4]:
# escolhas = []
# for d in data:
#     for i in d:
#         l = list( set( [ disc[ : -2 ] for disc in d[ i ][ 'formularios' ].keys() ] ) )
#         l.sort()
#         escolhas.append( l )

# j = json.dumps( escolhas, indent = 3 )
# print( j )

In [5]:
print( len( turmas ), turmas )
print( len( cods ), cods )

71 ['EST5102,1', 'EST5106,1', 'EST5507,1', 'EST5514,1', 'EST5519,1', 'EST5521,1', 'EST5802,1', 'EST5804,1', 'MAI5002,1', 'MAI5003,1', 'MAI5014,1', 'MAI5032,1', 'PRG0027,1', 'SMA5802,1', 'SMA5934,1', 'SME0110,1', 'SME0110,2', 'SME0123,1', 'SME0123,2', 'SME0130,1', 'SME0142,1', 'SME0142,2', 'SME0206,1', 'SME0211,1', 'SME0212,1', 'SME0213,1', 'SME0220,1', 'SME0240,1', 'SME0241,1', 'SME0245,1', 'SME0250,1', 'SME0252,1', 'SME0262,1', 'SME0284,1', 'SME0285,1', 'SME0300,1', 'SME0300,2', 'SME0306,1', 'SME0306,2', 'SME0320,1', 'SME0320,2', 'SME0320,3', 'SME0320,4', 'SME0332,1', 'SME0332,2', 'SME0334,1', 'SME0341,1', 'SME0341,2', 'SME0510,1', 'SME0800,1', 'SME0805,1', 'SME0806,1', 'SME0808,1', 'SME0809,1', 'SME0810,1', 'SME0814,1', 'SME0815,1', 'SME0820,1', 'SME0822,1', 'SME0823,1', 'SME0824,1', 'SME0828,1', 'SME0870,1', 'SME0871,1', 'SME0880,1', 'SME0881,1', 'SME5827,1', 'SME5873,1', 'SME5962,1', 'SME5975,1', 'SME5978,1']
61 ['EST5102', 'EST5106', 'EST5507', 'EST5514', 'EST5519', 'EST5521', 'ES

# Uma Rede das Disciplinas

Iremos construir um grafo onde

- os vértices serão as disciplinas

- as arestas terão capacidades correspondentes ao número de docentes que escolheu ambas as disciplinas ligadas

- o cálculo das capacidades podem levar em conta outros fatores

In [6]:
A = np.zeros( ( len( cods ), len( cods ) ) )

for datum in data:
    for doc in datum:
        form = datum[ doc ][ 'formularios' ]
        
        curr_cods = [ t[ : -2 ] for t in form.keys() ]
        curr_cods = list( set( curr_cods ) )
        curr_cods.sort()
        
        for i, cod_a in enumerate( curr_cods ):
            for j, cod_b in enumerate( curr_cods ):
                if i > j:
                    idx_i = cods.index( cod_a )
                    idx_j = cods.index( cod_b )

                    A[ idx_i, idx_j ] += 1
                    A[ idx_j, idx_i ] += 1


In [7]:
G = nx.Graph()

for c in cods:
    G.add_node( c )

for i, n_a in enumerate( G.nodes ):
    for j, n_b in enumerate( G.nodes ):
        if i > j:
            idx_i = cods.index( n_a )
            idx_j = cods.index( n_b )

            if A[ idx_i, idx_j ] > 0:
                G.add_edge( n_a, n_b, capacity = A[ idx_i, idx_j ] )

print( G )
print( A.mean() )

Graph with 61 nodes and 526 edges
0.48696586938994896


In [8]:
pos = nx.arf_layout( G )

In [9]:
def plota_grafo( G, pos = None, marker_size = 20 ):

    if pos is None:
        pos = nx.arf_layout( G )

    x = []
    y = []

    for e in G.edges:
        x.append( pos[ e[ 0 ] ][ 0 ] )
        x.append( pos[ e[ 1 ] ][ 0 ] )
        x.append( None )
        y.append( pos[ e[ 0 ] ][ 1 ] )
        y.append( pos[ e[ 1 ] ][ 1 ] )
        y.append( None )
    e_trace = go.Scatter(
        mode = 'lines',
        x = x,
        y = y,
        line = {
            'width': 0.5,
            'color': 'rgba( 0, 0, 0, 0.25 )'
        },
        hoverinfo = 'text'
    )
    e_trace.text = [ '' for p in pos ]

    n_trace = go.Scatter(
        mode = 'markers',
        x = [ pos[ p ][ 0 ] for p in pos ],
        y = [ pos[ p ][ 1 ] for p in pos ],
        marker = {
            'size': marker_size,
            'line': {
                'width': 0.25
            }
        },
        hoverinfo = 'text'
    )
    n_trace.text = [ p for p in pos ]

    fig = go.Figure(
        data = [ e_trace, n_trace ],
        layout = {
            'width': 600,
            'height': 600,
            'showlegend': False
        }
    )

    fig.show()

    return fig

In [10]:
_ = plota_grafo( G, nx.spring_layout( G ) )

In [11]:
_ = plota_grafo( G, nx.bfs_layout( G, 'SME0300' ) )

In [12]:
_ = plota_grafo( G, nx.forceatlas2_layout( G ) )

In [13]:
_ = plota_grafo( G, nx.shell_layout( G ) )

In [14]:
_ = plota_grafo( G, nx.kamada_kawai_layout( G ) )

In [15]:
_ = plota_grafo( G, nx.spectral_layout( G ), marker_size = 10 )

In [16]:
h_adj = go.Heatmap( z = A )

fig = go.Figure( data = [ h_adj ], layout = { 'width': 600, 'height': 600 } )
fig.show()

In [17]:
import sklearn.cluster as sc

k = 5
kmeans = sc.KMeans( n_clusters = k, init = 'random' ).fit( A )

clusters = { g: [] for g in range( k ) }
for i, c in enumerate( kmeans.labels_ ):
    clusters[ c ].append( cods[ i ] )

for c in clusters:
    print( clusters[ c ] )

['MAI5032', 'SME0110', 'SME0211', 'SME0212', 'SME0213', 'SME0284', 'SME0285', 'SME0332', 'SME0510', 'SME5962']
['SME0206', 'SME0241', 'SME0300', 'SME0306', 'SME0334']
['SME0123', 'SME0220', 'SME0320', 'SME0800', 'SME0820', 'SME0828', 'SME5975']
['MAI5002', 'MAI5014', 'SME0142', 'SME0240', 'SME0245', 'SME0250', 'SME0252', 'SME0341', 'SME5827', 'SME5873']
['EST5102', 'EST5106', 'EST5507', 'EST5514', 'EST5519', 'EST5521', 'EST5802', 'EST5804', 'MAI5003', 'PRG0027', 'SMA5802', 'SMA5934', 'SME0130', 'SME0262', 'SME0805', 'SME0806', 'SME0808', 'SME0809', 'SME0810', 'SME0814', 'SME0815', 'SME0822', 'SME0823', 'SME0824', 'SME0870', 'SME0871', 'SME0880', 'SME0881', 'SME5978']


In [18]:
import sklearn.decomposition as sd

pca = sd.PCA( n_components = 2 )
PA = pca.fit_transform( A )

In [19]:
_= plota_grafo( G, { n: PA[ i, : ] for i, n in enumerate( cods ) }, marker_size = 10 )